In [1]:
import nibabel as nib
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tqdm import tqdm
from glob import glob
import gc
import cv2
import matplotlib.pyplot as plt
import albumentations as albu
from PIL import Image
from sklearn.utils import shuffle
import math
import zipfile
%matplotlib inline

In [7]:
train_csv = pd.read_csv('./Train/name_mapping.csv')
val_csv = pd.read_csv('./Val/name_mapping_validation_data.csv')

In [8]:
train_csv.head()

,BraTS_2020_subject_ID
0,BraTS20_Training_001
1,BraTS20_Training_002
2,BraTS20_Training_003
3,BraTS20_Training_004
4,BraTS20_Training_005


In [9]:
val_csv.head()

,BraTS_2020_subject_ID
0,BraTS20_Validation_001
1,BraTS20_Validation_002
2,BraTS20_Validation_003
3,BraTS20_Validation_004
4,BraTS20_Validation_005


In [19]:
train_data= './Train/'+train_csv['BraTS_2020_subject_ID']+'/'+train_csv['BraTS_2020_subject_ID']+'_flair.nii.gz'
train_label= './Train/'+train_csv['BraTS_2020_subject_ID']+train_csv['BraTS_2020_subject_ID']+'_seg.nii.gz'
val_data= './Train/'+val_csv['BraTS_2020_subject_ID']+'/'+val_csv['BraTS_2020_subject_ID']+'_flair.nii.gz'
val_label= './Train/'+val_csv['BraTS_2020_subject_ID']+val_csv['BraTS_2020_subject_ID']+'_seg.nii.gz'

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, x_dir, y_dir,augmentation=True):
        super().__init__()
        
        self.augmentation = augmentation
        self.x_img = x_dir
        self.y_img = y_dir   

    def __len__(self):
        return len(self.x_img)

    def __getitem__(self, idx):
        x_img = self.x_img[idx]
        y_img = self.y_img[idx]
        # Read an image with OpenCV
        x_img = dcm.read_file(x_img)
        y_img =  imread(y_img)

        x_img=read_dicom(x_img,400,100)
        x_img=np.transpose(x_img,(2,0,1))
        x_img=x_img.astype(np.float32)

        y_img = resize(y_img, (512, 512))*255
        color_im = np.zeros([512, 512, 2])
        for i in range(1,3):
            encode_ = to_binary(y_img, i*1.0, i*1.0) * 255
            color_im[:, :, i-1] = encode_
        color_im = np.transpose(color_im,(2,0,1))
        # Data Augmentation
        if self.augmentation:
            img, mask = augment_imgs_and_masks(x_img, color_im, rot_factor, scale_factor, trans_factor, flip)

        return img, mask,y_img
